17102022

Бруцкий-Стемпковский

v. 01


Подготовить данные для этапа моделирования (обучения)
    
1. Анализ данных, необходимые корректировки. 
2. Обработать пропуски. 
3. Оценить выбросы. 
4. Корреляция. 
5. Тест на нормальность распределения.
6. Масштабировать и стандартизировать данные.
7. Разделите данные на тренировачную и тестовую выборку.
8. При обработке пропусков/выбросов/скалировании тестовую и тренировачную выборку обрабатывать отдельно.

Необходимые импорты библиотек.

In [165]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
import pandas_profiling

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

Прочитаем фрейм. Переименуем колонки.

In [166]:
columns = [ 'id', 'gender', 'age', 
            'marital_status', 'job', 
            'sum_of_credit', 'credit_term',
            'tarrif_id', 'score_shk', 'edu', 
            'region', 'income', 'credit_count', 
            'overdue_credit_count', 'flag'
            ]
data = pd.read_csv('credit_train.csv', names=columns, skiprows=1, encoding='utf8', sep=';', decimal=",")
data.set_index("id", inplace=True)
data.head()

,gender,age,marital_status,job,sum_of_credit,credit_term,tarrif_id,score_shk,edu,region,income,credit_count,overdue_credit_count,flag
id,,,,,,,,,,,,,,
1,M,NaN,NaN,UMN,59998.00,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
2,F,NaN,MAR,UMN,10889.00,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
3,M,32.0,MAR,SPC,10728.00,12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
4,F,27.0,NaN,SPC,12009.09,12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
5,M,45.0,NaN,SPC,NaN,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


Проверим информацию о строках.

In [167]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170746 entries, 1 to 170746
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job                   170746 non-null  object 
 4   sum_of_credit         170744 non-null  float64
 5   credit_term           170746 non-null  int64  
 6   tarrif_id             170746 non-null  object 
 7   score_shk             170739 non-null  float64
 8   edu                   170741 non-null  object 
 9   region                170554 non-null  object 
 10  income                170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  flag                  170746 non-null  int64  
dtypes: float64(6), int64(2), object(6)
memory usage: 19.

Фрейм содержит 14 признаков. Как числовые, так и категориальные признаки. Целевой показатель (функция от всех остальных аргументов) зашит в последней колонке 'flag'.

Большое количество выбросов сдержится в категориях о количестве имеющихся непогашенных и просроченных кредитов. От выбросов в остальных колонка просто избавимся.

Можно было бы заменить значения отдельных столбцов на среднее для числовых значений (применим это для столбиков с кредитами), либо заменить на самое часто встречающееся для категориальных значений.

In [168]:
drop_columns = ['gender', 'age', 'marital_status', 'job', 'sum_of_credit', 'credit_term',
                'tarrif_id', 'score_shk', 'edu', 'region', 'income', 'flag']
data.dropna(subset=drop_columns, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170543 entries, 8 to 170746
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170543 non-null  object 
 1   age                   170543 non-null  float64
 2   marital_status        170543 non-null  object 
 3   job                   170543 non-null  object 
 4   sum_of_credit         170543 non-null  float64
 5   credit_term           170543 non-null  int64  
 6   tarrif_id             170543 non-null  object 
 7   score_shk             170543 non-null  float64
 8   edu                   170543 non-null  object 
 9   region                170543 non-null  object 
 10  income                170543 non-null  float64
 11  credit_count          161331 non-null  float64
 12  overdue_credit_count  161331 non-null  float64
 13  flag                  170543 non-null  int64  
dtypes: float64(6), int64(2), object(6)
memory usage: 19.

Было удалено ~200 записей, что вполне приемлемо при общем количестве в 170 000.

По результатам анализа видно следующее:

1. Колонки 'gender', 'age', 'marital_status', 'job', 'credit_term', 'tarrif_id', 'edu', 'income', 'flag'
очищены от испорченных данных. То есть их тип соотвествует ожидаемому, нет выбросов.

2. Колонки 'sum_of_credit' и 'score_shk' имеют большое число пропусков, но заменять пропуски мы будем после разбивки на тренировочоный и тестовый сеты.

3. Колонка 'region' имеет 301 уникальное значение, представляющее собой как строковые записи (повторяются по своей сути, но не по значению), встречаются числовые значения (74 и 98), вероятно представляющие собой номера регионов (Челябинск и Санкт-Петербург соответсвенно).

Пока мы избавимся от колонки "region".

In [169]:
data_region = data["region"]
data.drop(["region"], axis=1, inplace=True)

Теперь заполним пропуски для обеих выборок в колонках о количестве кредитов, заменив их на среднее значение.

In [170]:
data["credit_count"].fillna(data["credit_count"].mean(), inplace=True)
data["overdue_credit_count"].fillna(data["overdue_credit_count"].mean(), inplace=True)

Для наглядного анализа данных применим библиотеку Pandas_profiling.

In [171]:
#pandas_profiling.ProfileReport(data)

In [172]:
data.describe()

,age,sum_of_credit,credit_term,score_shk,income,credit_count,overdue_credit_count,flag
count,170543.000000,170543.000000,170543.000000,170543.000000,170543.000000,170543.000000,170543.000000,170543.000000
mean,36.499845,26093.671464,10.980990,0.469446,40136.940508,2.107537,0.046061,0.176237
std,10.547081,16233.183098,3.537248,0.124255,25046.924003,1.728712,0.205710,0.381023
min,18.000000,2736.000000,3.000000,0.000000,5000.000000,0.000000,0.000000,0.000000
25%,28.000000,14908.000000,10.000000,0.379495,25000.000000,1.000000,0.000000,0.000000
50%,34.000000,21229.000000,10.000000,0.461559,35000.000000,2.000000,0.000000,0.000000
75%,43.000000,32068.000000,12.000000,0.552366,50000.000000,3.000000,0.000000,0.000000
max,71.000000,200000.000000,36.000000,1.128291,950000.000000,21.000000,3.000000,1.000000


Для дальнейшего анализа кодируем категориальные значения в числовые с помощью метода get_dammies. Оставшиеся категориальные солбцы удаляем, вместо них появятся новые числовые столбцы.

In [173]:
data = pd.concat([ data, 
                    pd.get_dummies(data['gender'], prefix="gender"),
                    pd.get_dummies(data['marital_status'], prefix="marital_status"),
                    pd.get_dummies(data['job'], prefix="job"),
                    pd.get_dummies(data['edu'], prefix="edu")],
                    axis=1)
data.drop(["gender", "marital_status", "job", "edu"], axis=1, inplace=True)

In [174]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170543 entries, 8 to 170746
Data columns (total 39 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   age                   170543 non-null  float64
 1   sum_of_credit         170543 non-null  float64
 2   credit_term           170543 non-null  int64  
 3   tarrif_id             170543 non-null  object 
 4   score_shk             170543 non-null  float64
 5   income                170543 non-null  float64
 6   credit_count          170543 non-null  float64
 7   overdue_credit_count  170543 non-null  float64
 8   flag                  170543 non-null  int64  
 9   gender_F              170543 non-null  uint8  
 10  gender_M              170543 non-null  uint8  
 11  marital_status_CIV    170543 non-null  uint8  
 12  marital_status_DIV    170543 non-null  uint8  
 13  marital_status_MAR    170543 non-null  uint8  
 14  marital_status_UNM    170543 non-null  uint8  
 15  

Тест Шапиро-Уилка

In [175]:
def normal_test(ser):
    stat, p = sp.stats.shapiro(ser)

    alpha = 0.05
    if p > alpha:
        return 'Похоже на нормальное распределение'
    else:
        return 'Не похоже на нормальное распределение'


In [176]:
for name in data.columns[:9]:
    print(name, normal_test(data[name]))

c:\Users\Николай\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


age Не похоже на нормальное распределение
sum_of_credit Не похоже на нормальное распределение
credit_term Не похоже на нормальное распределение


TypeError: ufunc 'divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [177]:
corr_matrix = data.corr()

C:\Users\Николай\AppData\Local\Temp\ipykernel_7792\962194556.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = data.corr()


In [178]:
corr_matrix

,age,sum_of_credit,credit_term,score_shk,income,credit_count,overdue_credit_count,flag,gender_F,gender_M,...,job_SPC,job_UMN,job_WOI,job_WRK,job_WRP,edu_ACD,edu_GRD,edu_PGR,edu_SCH,edu_UGR
age,1.000000,-0.096471,0.007131,-0.166144,-0.014842,1.118157e-01,7.520229e-03,-0.031196,0.125474,-0.125474,...,-0.157929,-0.001372,-0.004556,-0.009950,0.050190,0.002920,-0.060589,-0.004851,0.119628,-0.126568
sum_of_credit,-0.096471,1.000000,0.228409,0.054343,0.351380,1.539512e-02,-3.826026e-03,-0.072059,-0.033999,0.033999,...,-0.071839,0.066994,-0.004857,-0.013535,-0.004628,0.008477,0.178794,0.019074,-0.188456,0.019217
credit_term,0.007131,0.228409,1.000000,0.040412,0.007686,4.914018e-02,6.299802e-03,0.025792,-0.001873,0.001873,...,-0.004284,-0.011274,0.006527,0.000334,0.008886,-0.005095,0.009098,0.013551,-0.007704,-0.005545
score_shk,-0.166144,0.054343,0.040412,1.000000,-0.107384,-8.968154e-02,1.628305e-01,0.048825,0.012532,-0.012532,...,0.037298,-0.044455,0.004226,0.017509,-0.003622,-0.013574,-0.249142,-0.036329,0.223533,0.059213
income,-0.014842,0.351380,0.007686,-0.107384,1.000000,1.440034e-01,3.312977e-02,-0.023683,-0.202210,0.202210,...,-0.156368,0.126143,-0.007541,-0.007710,-0.001295,0.025608,0.214287,0.049345,-0.228828,0.021200
credit_count,0.111816,0.015395,0.049140,-0.089682,0.144003,1.000000e+00,1.773034e-01,0.034084,0.014413,-0.014413,...,-0.023931,0.026341,-0.000867,0.009296,0.014065,0.005650,0.030812,0.014504,-0.025246,-0.015325
overdue_credit_count,0.007520,-0.003826,0.006300,0.162831,0.033130,1.773034e-01,1.000000e+00,0.022254,-0.001388,0.001388,...,-0.011331,0.011290,0.001102,0.005333,-0.001146,0.001325,0.021411,0.007458,-0.019982,-0.004523
flag,-0.031196,-0.072059,0.025792,0.048825,-0.023683,3.408431e-02,2.225362e-02,1.000000,-0.044381,0.044381,...,-0.049172,0.011600,0.007448,0.001340,0.002191,0.003775,-0.082448,0.002257,0.078409,0.005739
gender_F,0.125474,-0.033999,-0.001873,0.012532,-0.202210,1.441327e-02,-1.387895e-03,-0.044381,1.000000,-1.000000,...,0.023363,-0.022490,0.002099,-0.018738,0.003628,-0.008714,0.075324,-0.007261,-0.063785,-0.020158
gender_M,-0.125474,0.033999,0.001873,-0.012532,0.202210,-1.441327e-02,1.387895e-03,0.044381,-1.000000,1.000000,...,-0.023363,0.022490,-0.002099,0.018738,-0.003628,0.008714,-0.075324,0.007261,0.063785,0.020158


Разделим сет на тренировочную и тестовую выборки в отношении 70/30 соответсвенно. Деление выполним с помощью библиотеки sklearn. Указанные параметры указывают на то, что размер тестовой выборки будет составлять 30 %, а повторное деление сета даст тот же результат, что и при первом запуске. Далее показатель random_state можно удалить и тогда деление будет осуществляться каим-то новым образом.

In [179]:
train, test = train_test_split(data, test_size=0.3, random_state = 1)

Для дальнейшей работы обновим список индекссов.

In [180]:
train.set_index(np.arange(len(train)), inplace=True)
test.set_index(np.arange(len(test)), inplace=True)

Выполним нормализацию сетов по некатегориальным значениям. Для этого разделим сет на две части.

In [183]:
st_columns = ["age", "sum_of_credit", "credit_term", "income", "credit_count", "overdue_credit_count"]

no_st_columns = list(data.columns)

for _ in st_columns:
    no_st_columns.remove(_)

train_st_copy = train[st_columns].copy()
test_st_copy = test[st_columns].copy()

train_no_st_st_copy = train[no_st_columns].copy()
test_no_st_copy = test[no_st_columns].copy()

scaler = StandardScaler()

d = scaler.fit_transform(train_st_copy)
train_st_copy = pd.DataFrame(d, columns = st_columns)

d = scaler.fit_transform(test_st_copy)
test_st_copy = pd.DataFrame(d, columns = st_columns)

"Склеим" обратно нормализованную и категориальные части сета между собой.

In [185]:
train_st = pd.concat([train_st_copy, train[no_st_columns]], axis=1)
test_st  = pd.concat([test_st_copy, test[no_st_columns]], axis=1)

Выделим из сетов колонки с "аргуменами" и "функцией".

In [186]:
y_train, x_train = train_st["flag"], train_st.drop(["flag"], axis=1)
y_test, x_test = test_st["flag"], test_st.drop(["flag"], axis=1)

Натренируем случайный лес.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 10,  max_depth=5,  random_state=21)
rfc.fit(x_train, y_train)
rfc.score (x_test, y_test)

Выполним AUC оценку.

In [ ]:
y_pred = rfc.predict_proba(x_test)[:, 1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)